In [1]:
import numpy as np
from numpy.random import randn
from numpy.random import randint
import os
import cv2
import pandas as pd
import gzip
from tensorflow.keras import backend as K
import tensorflow as tf 
from matplotlib import pyplot as plt
from keras.preprocessing.image import load_img
from keras.datasets.mnist import load_data
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from tensorflow.python.keras.utils import conv_utils

Using TensorFlow backend.


In [2]:
#epsilon set according to BIGGAN https://arxiv.org/pdf/1809.11096.pdf

def _l2normalizer(v, epsilon=1e-4):
    return v / (K.sum(v**2)**0.5 + epsilon)


def power_iteration(W, u, rounds=1):
    '''
    Accroding the paper, we only need to do power iteration one time.
    '''
    _u = u

    for i in range(rounds):
        _v = _l2normalizer(K.dot(_u, W))
        _u = _l2normalizer(K.dot(_v, K.transpose(W)))

    W_sn = K.sum(K.dot(_u, W) * _v)
    return W_sn, _u, _v

"""
Convolution 2D with spectral normalization
"""
class SNConv2D(Conv2D):
    def __init__(self, filters, spectral_normalization=True, **kwargs):
        self.spectral_normalization = spectral_normalization
        super(SNConv2D, self).__init__(filters, **kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.

        self.u = self.add_weight(name='u', shape=(1, self.filters),
                                 initializer='uniform', trainable=False)
        super(SNConv2D, self).build(input_shape)
        # Be sure to call this at the end

    def compute_spectral_normal(self, training=True):
        # Spectrally Normalized Weight
        if self.spectral_normalization:
            # Get kernel tensor shape [kernel_h, kernel_w, in_channels, out_channels]
            W_shape = self.kernel.shape.as_list()

            # Flatten the Tensor
            # [out_channels, N]
            W_mat = K.reshape(self.kernel, [W_shape[-1], -1])

            W_sn, u, v = power_iteration(W_mat, self.u)

            if training:
                # Update estimated 1st singular vector
                self.u.assign(u)

            return self.kernel / W_sn
        else:
            return self.kernel

    def call(self, inputs, training=None):

        outputs = K.conv2d(inputs,
                           self.compute_spectral_normal(training=training),
                           strides=self.strides, padding=self.padding,
                           data_format=self.data_format,
                           dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(outputs, self.bias,
                                 data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        return super(SNConv2D, self).compute_output_shape(input_shape)

"""
Transposed Convolution 2D with spectral normalization
"""
class SNConv2DTranspose(Conv2DTranspose):
    def __init__(self, spectral_normalization=True, **kwargs):
        self.spectral_normalization = spectral_normalization
        super(SNConv2DTranspose, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.u = self.add_weight(name='u', shape=(1, self.filters),
                                 initializer='uniform', trainable=False)
        super(SNConv2DTranspose, self).build(input_shape)
        # Be sure to call this at the end

    def compute_spectral_normal(self, training=True):
        # Spectrally Normalized Weight
        if self.spectral_normalization:
            # Get kernel tensor shape [kernel_h, kernel_w, in_channels, out_channels]
            W_shape = self.kernel.shape.as_list()

            # Flatten the Tensor
            # [out_channels, N]
            W_mat = K.reshape(self.kernel, [W_shape[-2], -1])

            W_sn, u, v = power_iteration(W_mat, self.u)

            if training:
                # Update estimated 1st singular vector
                self.u.assign(u)

            return self.kernel / W_sn
        else:
            return self.kernel

    def call(self, inputs, training=None):
        input_shape = K.shape(inputs)
        batch_size = input_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = input_shape[h_axis], input_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides
        out_pad_h = out_pad_w = None

        # Infer the dynamic output shape:
        out_height = conv_utils.deconv_output_length(
            height, kernel_h, self.padding, stride=stride_h)
        out_width = conv_utils.deconv_output_length(
            width, kernel_w, self.padding, stride=stride_w)
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)

        outputs = K.conv2d_transpose(
            inputs,
            self.compute_spectral_normal(training=training),
            output_shape,
            self.strides,
            padding=self.padding,
            data_format=self.data_format
        )

        if self.use_bias:
            outputs = K.bias_add(outputs, self.bias,
                                 data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        return super(SNConv2DTranspose, self).compute_output_shape(input_shape)

In [3]:
# define the standalone discriminator model
def define_discriminator(in_shape=(64,64,3)):
    model = Sequential()
    # normal
    model.add(SNConv2D(filters=128, kernel_size=4, padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 40x40
    model.add(SNConv2D(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 20x20
    model.add(SNConv2D(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 10x10
    model.add(SNConv2D(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 5x5
    model.add(SNConv2D(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 5x5 feature maps
    n_nodes = 128 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 128)))
    # upsample to 10x10
    model.add(SNConv2DTranspose(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 20x20
    model.add(SNConv2DTranspose(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 40x40
    model.add(SNConv2DTranspose(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 80x80
    model.add(SNConv2DTranspose(filters=128, kernel_size=4, strides=(2,2), padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    model.add(LeakyReLU(alpha=0.2))
    # output layer 80x80x3
    model.add(SNConv2D(3, kernel_size=4, activation='tanh', padding='same', kernel_initializer="orthogonal", spectral_normalization=True))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate ✬real✬ class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create ✬fake✬ class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

# create and save a p
def save_plot(examples, epoch, n=4):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i])
    # save plot to file
    filename = 'model_img/generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'models/generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

    
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=30, n_batch=32):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected ✬real✬ samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate ✬fake✬ examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator✬s error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            # evaluate the model performance, sometimes
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [9]:
directory = 'img_align_celeba/img_align_celeba/'
how_many = 150000
celeba_img = np.sort(os.listdir(directory))
dataset = celeba_img[:how_many]
img_shape = (178, 218, 3)

def load_dataset(dataset):
    X = []
    for i, myid in enumerate(dataset):
        image = load_img(directory + "/" + myid, target_size=img_shape[:2])
        image = (img_to_array(image) - 127.5) / 127.5 # from [0, 255] to [-1, 1]
        image = image[25:153, 45:173]
        image = cv2.resize(src=image, dsize=(64, 64))
        X.append(image)
    X = np.array(X)
    return(X)

In [10]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
#dataset = load_real_samples()
dataset = load_dataset(dataset)

In [12]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/4687, d1=0.317, d2=0.197 g=3.599
>1, 2/4687, d1=0.048, d2=0.104 g=4.064
>1, 3/4687, d1=0.489, d2=0.252 g=3.763
>1, 4/4687, d1=0.151, d2=0.168 g=3.695
>1, 5/4687, d1=0.386, d2=0.235 g=3.641
>1, 6/4687, d1=0.162, d2=0.590 g=5.233
>1, 7/4687, d1=0.263, d2=0.031 g=4.674
>1, 8/4687, d1=0.287, d2=0.197 g=4.782
>1, 9/4687, d1=0.312, d2=0.142 g=4.600
>1, 10/4687, d1=0.112, d2=0.131 g=4.816
>1, 11/4687, d1=0.262, d2=0.066 g=3.940
>1, 12/4687, d1=0.127, d2=0.639 g=5.870
>1, 13/4687, d1=0.149, d2=0.006 g=6.811
>1, 14/4687, d1=1.255, d2=0.184 g=2.791
>1, 15/4687, d1=0.183, d2=0.422 g=3.298
>1, 16/4687, d1=0.263, d2=0.065 g=3.769
>1, 17/4687, d1=0.252, d2=0.190 g=3.469
>1, 18/4687, d1=0.097, d2=0.084 g=3.392
>1, 19/4687, d1=0.229, d2=0.183 g=3.007
>1, 20/4687, d1=0.076, d2=0.098 g=3.716
>1, 21/4687, d1=0.112, d2=0.169 g=4.184
>1, 22/4687, d1=0.407, d2=0.085 g=3.336
>1, 23/4687, d1=0.169, d2=0.226 g=3.811
>1, 24/4687, d1=0.062, d2=0.324 g=5.857
>1, 25/4687, d1=0.273, d2=0.036 g=4.559
>1, 26/46

>1, 205/4687, d1=0.345, d2=0.158 g=3.315
>1, 206/4687, d1=0.189, d2=0.631 g=3.403
>1, 207/4687, d1=0.414, d2=0.233 g=3.226
>1, 208/4687, d1=0.867, d2=0.366 g=3.322
>1, 209/4687, d1=0.317, d2=0.111 g=3.293
>1, 210/4687, d1=0.100, d2=0.177 g=3.014
>1, 211/4687, d1=0.220, d2=0.394 g=3.507
>1, 212/4687, d1=0.227, d2=0.236 g=3.565
>1, 213/4687, d1=0.203, d2=0.137 g=3.789
>1, 214/4687, d1=0.315, d2=0.182 g=2.961
>1, 215/4687, d1=0.071, d2=0.192 g=3.203
>1, 216/4687, d1=0.382, d2=0.173 g=3.280
>1, 217/4687, d1=0.183, d2=0.246 g=3.760
>1, 218/4687, d1=0.423, d2=0.232 g=4.226
>1, 219/4687, d1=0.050, d2=0.090 g=3.949
>1, 220/4687, d1=0.056, d2=0.111 g=3.726
>1, 221/4687, d1=0.049, d2=0.119 g=3.479
>1, 222/4687, d1=0.211, d2=0.382 g=4.951
>1, 223/4687, d1=0.628, d2=0.121 g=2.961
>1, 224/4687, d1=0.186, d2=0.580 g=4.502
>1, 225/4687, d1=0.663, d2=0.086 g=3.900
>1, 226/4687, d1=0.200, d2=0.270 g=3.714
>1, 227/4687, d1=0.335, d2=0.097 g=3.937
>1, 228/4687, d1=0.232, d2=0.441 g=4.713
>1, 229/4687, d1

>1, 405/4687, d1=0.362, d2=0.088 g=4.012
>1, 406/4687, d1=0.499, d2=0.275 g=2.928
>1, 407/4687, d1=0.136, d2=0.127 g=3.035
>1, 408/4687, d1=0.450, d2=0.376 g=3.134
>1, 409/4687, d1=0.060, d2=0.151 g=3.926
>1, 410/4687, d1=0.120, d2=0.173 g=4.633
>1, 411/4687, d1=0.127, d2=0.210 g=4.711
>1, 412/4687, d1=0.588, d2=0.280 g=4.645
>1, 413/4687, d1=0.242, d2=0.080 g=3.787
>1, 414/4687, d1=0.590, d2=0.523 g=3.811
>1, 415/4687, d1=0.052, d2=0.067 g=4.675
>1, 416/4687, d1=0.546, d2=0.399 g=3.864
>1, 417/4687, d1=0.263, d2=0.057 g=3.719
>1, 418/4687, d1=0.036, d2=0.105 g=3.191
>1, 419/4687, d1=0.060, d2=0.246 g=3.800
>1, 420/4687, d1=0.384, d2=0.088 g=3.770
>1, 421/4687, d1=0.060, d2=0.077 g=3.574
>1, 422/4687, d1=0.513, d2=0.511 g=4.237
>1, 423/4687, d1=0.376, d2=0.070 g=4.120
>1, 424/4687, d1=0.284, d2=0.492 g=4.814
>1, 425/4687, d1=0.111, d2=0.213 g=4.836
>1, 426/4687, d1=0.754, d2=0.270 g=3.382
>1, 427/4687, d1=0.181, d2=0.227 g=3.961
>1, 428/4687, d1=0.420, d2=0.189 g=3.822
>1, 429/4687, d1

>1, 605/4687, d1=0.484, d2=0.288 g=2.551
>1, 606/4687, d1=0.369, d2=0.445 g=3.271
>1, 607/4687, d1=0.248, d2=0.097 g=2.959
>1, 608/4687, d1=0.211, d2=0.782 g=3.229
>1, 609/4687, d1=0.243, d2=0.133 g=3.021
>1, 610/4687, d1=0.350, d2=0.405 g=2.851
>1, 611/4687, d1=0.246, d2=0.119 g=3.116
>1, 612/4687, d1=0.324, d2=0.292 g=3.406
>1, 613/4687, d1=0.198, d2=0.269 g=3.788
>1, 614/4687, d1=0.341, d2=0.125 g=3.276
>1, 615/4687, d1=0.206, d2=0.347 g=3.276
>1, 616/4687, d1=0.281, d2=0.499 g=3.654
>1, 617/4687, d1=0.316, d2=0.108 g=3.394
>1, 618/4687, d1=0.374, d2=0.577 g=3.379
>1, 619/4687, d1=0.336, d2=0.136 g=3.575
>1, 620/4687, d1=0.490, d2=0.125 g=3.058
>1, 621/4687, d1=0.177, d2=0.218 g=2.831
>1, 622/4687, d1=0.348, d2=0.345 g=3.414
>1, 623/4687, d1=0.499, d2=0.186 g=2.786
>1, 624/4687, d1=0.584, d2=0.489 g=2.415
>1, 625/4687, d1=0.191, d2=0.182 g=3.479
>1, 626/4687, d1=0.244, d2=0.495 g=4.346
>1, 627/4687, d1=0.638, d2=0.067 g=2.743
>1, 628/4687, d1=0.219, d2=0.249 g=2.592
>1, 629/4687, d1

>1, 805/4687, d1=0.540, d2=0.109 g=3.219
>1, 806/4687, d1=0.379, d2=0.336 g=2.710
>1, 807/4687, d1=0.014, d2=0.166 g=4.047
>1, 808/4687, d1=0.216, d2=0.048 g=4.326
>1, 809/4687, d1=0.057, d2=0.076 g=3.572
>1, 810/4687, d1=0.473, d2=0.408 g=3.983
>1, 811/4687, d1=0.153, d2=0.048 g=4.023
>1, 812/4687, d1=0.333, d2=0.181 g=3.477
>1, 813/4687, d1=0.480, d2=0.150 g=2.990
>1, 814/4687, d1=0.280, d2=0.455 g=3.772
>1, 815/4687, d1=0.142, d2=0.325 g=4.714
>1, 816/4687, d1=0.505, d2=0.146 g=4.174
>1, 817/4687, d1=0.232, d2=0.533 g=5.123
>1, 818/4687, d1=0.270, d2=0.020 g=4.522
>1, 819/4687, d1=0.191, d2=0.128 g=3.690
>1, 820/4687, d1=0.215, d2=0.384 g=4.799
>1, 821/4687, d1=0.269, d2=0.026 g=4.273
>1, 822/4687, d1=0.442, d2=1.276 g=7.530
>1, 823/4687, d1=1.229, d2=0.004 g=4.965
>1, 824/4687, d1=0.713, d2=0.119 g=2.174
>1, 825/4687, d1=0.075, d2=0.469 g=1.938
>1, 826/4687, d1=0.265, d2=0.245 g=2.595
>1, 827/4687, d1=0.143, d2=0.131 g=2.987
>1, 828/4687, d1=0.168, d2=0.142 g=2.650
>1, 829/4687, d1

...

>10, 3829/4687, d1=0.324, d2=0.165 g=3.831
>10, 3830/4687, d1=0.158, d2=0.096 g=3.748
>10, 3831/4687, d1=0.239, d2=0.176 g=3.335
>10, 3832/4687, d1=0.086, d2=0.051 g=3.024
>10, 3833/4687, d1=0.169, d2=0.135 g=2.611
>10, 3834/4687, d1=0.101, d2=0.286 g=4.332
>10, 3835/4687, d1=0.069, d2=0.048 g=4.754
>10, 3836/4687, d1=0.204, d2=0.154 g=3.578
>10, 3837/4687, d1=0.214, d2=0.333 g=4.003
>10, 3838/4687, d1=0.089, d2=0.044 g=4.147
>10, 3839/4687, d1=0.053, d2=0.055 g=3.915
>10, 3840/4687, d1=0.100, d2=0.111 g=4.231
>10, 3841/4687, d1=0.077, d2=0.025 g=4.138
>10, 3842/4687, d1=0.017, d2=0.091 g=3.867
>10, 3843/4687, d1=0.206, d2=0.132 g=3.607
>10, 3844/4687, d1=0.056, d2=0.199 g=4.872
>10, 3845/4687, d1=0.373, d2=0.135 g=4.375
>10, 3846/4687, d1=0.473, d2=0.234 g=4.516
>10, 3847/4687, d1=0.248, d2=0.100 g=5.195
>10, 3848/4687, d1=0.460, d2=0.044 g=3.410
>10, 3849/4687, d1=0.083, d2=0.341 g=4.719
>10, 3850/4687, d1=0.168, d2=0.021 g=4.927
>10, 3851/4687, d1=0.276, d2=0.088 g=3.696
>10, 3852/4

>10, 4021/4687, d1=0.322, d2=0.144 g=4.073
>10, 4022/4687, d1=0.000, d2=0.090 g=4.476
>10, 4023/4687, d1=0.424, d2=0.044 g=2.944
>10, 4024/4687, d1=0.302, d2=0.585 g=4.230
>10, 4025/4687, d1=0.164, d2=0.018 g=4.908
>10, 4026/4687, d1=0.294, d2=0.133 g=4.380
>10, 4027/4687, d1=0.464, d2=0.095 g=3.108
>10, 4028/4687, d1=0.147, d2=0.221 g=3.646
>10, 4029/4687, d1=0.411, d2=0.183 g=4.064
>10, 4030/4687, d1=0.179, d2=0.092 g=4.015
>10, 4031/4687, d1=0.237, d2=0.095 g=3.018
>10, 4032/4687, d1=0.001, d2=0.260 g=3.853
>10, 4033/4687, d1=0.187, d2=0.070 g=3.926
>10, 4034/4687, d1=0.123, d2=0.051 g=3.494
>10, 4035/4687, d1=0.251, d2=0.260 g=3.975
>10, 4036/4687, d1=0.155, d2=0.059 g=4.093
>10, 4037/4687, d1=0.101, d2=0.169 g=4.311
>10, 4038/4687, d1=0.034, d2=0.176 g=4.620
>10, 4039/4687, d1=0.124, d2=0.039 g=3.949
>10, 4040/4687, d1=0.223, d2=0.096 g=3.224
>10, 4041/4687, d1=0.052, d2=0.397 g=3.670
>10, 4042/4687, d1=0.445, d2=0.076 g=3.666
>10, 4043/4687, d1=0.233, d2=0.493 g=5.521
>10, 4044/4

>10, 4213/4687, d1=0.265, d2=0.228 g=3.728
>10, 4214/4687, d1=0.372, d2=0.290 g=5.027
>10, 4215/4687, d1=0.270, d2=0.069 g=4.052
>10, 4216/4687, d1=0.114, d2=0.117 g=3.915
>10, 4217/4687, d1=0.168, d2=0.053 g=3.585
>10, 4218/4687, d1=0.065, d2=0.222 g=4.276
>10, 4219/4687, d1=0.039, d2=0.048 g=3.499
>10, 4220/4687, d1=0.224, d2=0.362 g=4.583
>10, 4221/4687, d1=0.087, d2=0.032 g=5.258
>10, 4222/4687, d1=0.450, d2=0.322 g=4.081
>10, 4223/4687, d1=0.203, d2=0.100 g=3.893
>10, 4224/4687, d1=0.390, d2=0.169 g=3.708
>10, 4225/4687, d1=0.006, d2=0.033 g=4.109
>10, 4226/4687, d1=0.560, d2=0.255 g=3.517
>10, 4227/4687, d1=0.164, d2=0.168 g=4.349
>10, 4228/4687, d1=0.471, d2=0.074 g=2.505
>10, 4229/4687, d1=0.108, d2=0.752 g=5.868
>10, 4230/4687, d1=0.639, d2=0.030 g=4.370
>10, 4231/4687, d1=0.644, d2=0.266 g=2.562
>10, 4232/4687, d1=0.122, d2=0.260 g=3.696
>10, 4233/4687, d1=0.503, d2=0.219 g=3.399
>10, 4234/4687, d1=0.078, d2=0.200 g=3.953
>10, 4235/4687, d1=0.414, d2=0.089 g=3.431
>10, 4236/4

>10, 4405/4687, d1=0.223, d2=0.105 g=3.296
>10, 4406/4687, d1=0.362, d2=0.592 g=3.282
>10, 4407/4687, d1=0.253, d2=0.037 g=3.814
>10, 4408/4687, d1=0.145, d2=0.054 g=3.293
>10, 4409/4687, d1=0.570, d2=0.243 g=2.630
>10, 4410/4687, d1=0.173, d2=0.204 g=3.287
>10, 4411/4687, d1=0.182, d2=0.072 g=3.617
>10, 4412/4687, d1=0.015, d2=0.032 g=3.878
>10, 4413/4687, d1=0.026, d2=0.153 g=3.409
>10, 4414/4687, d1=0.315, d2=0.392 g=3.817
>10, 4415/4687, d1=0.221, d2=0.133 g=3.839
>10, 4416/4687, d1=0.189, d2=0.045 g=3.401
>10, 4417/4687, d1=0.367, d2=0.105 g=2.677
>10, 4418/4687, d1=0.063, d2=0.108 g=2.733
>10, 4419/4687, d1=0.072, d2=0.125 g=3.168
>10, 4420/4687, d1=0.236, d2=0.124 g=2.773
>10, 4421/4687, d1=0.150, d2=0.272 g=3.651
>10, 4422/4687, d1=0.295, d2=0.131 g=3.138
>10, 4423/4687, d1=0.557, d2=0.187 g=3.275
>10, 4424/4687, d1=0.186, d2=0.160 g=3.269
>10, 4425/4687, d1=0.217, d2=0.137 g=3.507
>10, 4426/4687, d1=0.226, d2=0.077 g=2.734
>10, 4427/4687, d1=0.195, d2=0.318 g=3.528
>10, 4428/4

>10, 4597/4687, d1=0.279, d2=0.029 g=3.455
>10, 4598/4687, d1=0.077, d2=0.080 g=2.894
>10, 4599/4687, d1=0.094, d2=0.208 g=3.916
>10, 4600/4687, d1=0.311, d2=0.118 g=3.454
>10, 4601/4687, d1=0.387, d2=0.205 g=3.791
>10, 4602/4687, d1=0.363, d2=0.142 g=3.398
>10, 4603/4687, d1=0.034, d2=0.051 g=4.011
>10, 4604/4687, d1=0.043, d2=0.051 g=4.078
>10, 4605/4687, d1=0.061, d2=0.244 g=4.415
>10, 4606/4687, d1=0.233, d2=0.073 g=4.059
>10, 4607/4687, d1=0.279, d2=0.648 g=5.566
>10, 4608/4687, d1=0.762, d2=0.052 g=3.561
>10, 4609/4687, d1=0.229, d2=0.338 g=3.455
>10, 4610/4687, d1=0.355, d2=0.253 g=3.091
>10, 4611/4687, d1=0.171, d2=0.284 g=3.649
>10, 4612/4687, d1=0.050, d2=0.042 g=3.877
>10, 4613/4687, d1=0.282, d2=0.187 g=3.406
>10, 4614/4687, d1=0.087, d2=0.240 g=3.867
>10, 4615/4687, d1=0.249, d2=0.047 g=3.122
>10, 4616/4687, d1=0.031, d2=0.199 g=4.205
>10, 4617/4687, d1=0.436, d2=0.126 g=3.398
>10, 4618/4687, d1=0.284, d2=0.326 g=4.371
>10, 4619/4687, d1=0.349, d2=0.143 g=3.681
>10, 4620/4

In [13]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# create and save a plot of generated images
def save_plot(examples, n):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :])
    pyplot.show()


In [14]:
# load model
model = load_model('models/generator_model_010.h5')
# generate images
latent_points = generate_latent_points(100, 100)
# generate images
X = model.predict(latent_points)
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 4)

ValueError: Unknown layer: SNConv2DTranspose